In [ ]:
import os
import pandas as pd
import tensorqtl
from tensorqtl import genotypeio, post
import rpgQTL
import sys

In [ ]:
### main output directory
out_dir = "./out_dir"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
### read genotype and expression data
plink_prefix_path = '../data/GTEX_download/GTEx_Analysis_v8_genotype/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.MAF01'
expression_bed = '../data/GTEX_download/GTEx_Analysis_v8_eQTL_expression_matrices/Liver.v8.normalized_expression.bed.gz'
covariates_file = '../data/GTEX_download/GTEx_Analysis_v8_eQTL_covariates/Liver.v8.covariates.txt'

phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T
pr = genotypeio.PlinkReader(plink_prefix_path, select_samples=phenotype_df.columns)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

In [ ]:
### RPG file
rpg_df = "./Liver.rpg_gtex"

In [ ]:
### nominal run
rpgQTL.run_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df,
                   rpg_df, l_window=2000000, s_window=0, NonHiCType='remove',
                   output_dir=out_dir, prefix="RPG")

In [ ]:
### permutation run
egenes_df = rpgQTL.run_permutation(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, 
                                   rpg_df, l_window=2000000, s_window=0, NonHiCType='remove', seed=123456)

In [ ]:
#### calculate q-values
post.calculate_qvalues(egenes_df, fdr=0.05, qvalue_lambda=0.85)
egenes_df.to_csv("%s/RPG.egenes.tsv" % out_dir, index=True)

In [ ]:
### significant pairs
pairs_df = post.get_significant_pairs(egenes_df, "%s/%s" % (out_dir, prefix))
pairs_df.to_csv("%s/RPG.sig_pairs.tsv" % out_dir, index=False)

In [ ]:
### independent eQTL
indep_df = rpgQTL.run_independent(genotype_df, variant_df, egenes_df, phenotype_df, phenotype_pos_df, covariates_df, 
                                  rpg_df, l_window=2000000, s_window=0, NonHiCType='remove', seed=123456)
indep_df.to_csv("%s/RPG.indep.tsv" % out_dir, index=False)